In [9]:
# Load library
import pandas as pd
import numpy as np
import os
import re

In [28]:
# Create result files -> in csv format
LOG_DIR = './save/'
RESULT_DIR = './result/'
EPOCH = 100

top_accuracy_df = pd.DataFrame(columns=['model', 'gpu', 'dataset', 'top-1 accuracy'])
loss_df = pd.DataFrame(columns=['Model_Name', 'GPU_Type', 'Dataset', 'Epoch', 'Val_Loss'])
for experiment in os.listdir(LOG_DIR):
    if experiment != 'resnet-Base--CIFAR10-LR[0.001]-Seed0-V100': 
        continue
    experiment_dir = os.path.join(LOG_DIR, experiment)
    if os.path.isdir(experiment_dir):
        model, args = experiment.strip().split('--')
        args = args.strip().split('-')
        dataset = 'T-IMNET' if args[0] == 'T' else args[0]
        gpu = args[-1]
        # Get Top-1 Accuracy
        with open(experiment_dir + '/history.csv', "r") as f:
            last_line = f.readlines()[-1]
            top_acc = float(re.findall("\d+\.\d+", last_line)[0])
            new_row = {'model':model, 
                       'gpu':gpu, 
                       'dataset':dataset, 
                       'top-1 accuracy':top_acc}
            top_accuracy_df = top_accuracy_df.append(new_row, ignore_index=True)
        # Get Validation Loss in each Epoch
        log_df = pd.read_csv(experiment_dir + '/log.csv')
        for epoch, row in log_df.iterrows():
            new_row = {'Model_Name':model, 
                       'GPU_Type':gpu, 
                       'Dataset':dataset, 
                       'Epoch':epoch,
                       'Val_Loss':row['V Loss']}    
            loss_df = loss_df.append(new_row, ignore_index=True)

In [22]:
top_accuracy_df.to_csv('result/accuracy.csv', index=False) 
loss_df.to_csv('result/loss.csv', index=False)